In [ ]:
import scanpy as sc
import squidpy as sq
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import pickle as pkl

In [ ]:
import sys
sys.path.append('..')
from util import load_original_data

In [ ]:
# the atlas adata
q = load_original_data()

# load the finegrained labels obs
qobs = pd.read_csv('../EsoAtlas_july182022_obs_w_fine_grain_celltypes.csv', index_col=0)

# getting into the right order
qobs = qobs.loc[q.obs.index,:]

# copying it over
q.obs = qobs

# drop cells without a fine grained label
qobs2 = q.obs.dropna(subset=['Fine_Grained_Label'])

q = q[qobs2.index]

In [3]:
diagstr = 'T'
qsub = q[q.obs.diagnosis == diagstr].copy()

In [ ]:
qsub.obs.Fine_Grained_Label.value_counts()

In [ ]:
qsub[qsub.obs.Fine_Grained_Label == 'Macrophages', "CXCL8"].X.toarray().mean()

In [ ]:
qsub[qsub.obs.Fine_Grained_Label == 'Neutrophils', "CXCR2"].X#.toarray().mean()

In [ ]:
del q, qobs, qobs2

In [5]:
# first normalize the counts
sc.pp.normalize_total(qsub, target_sum=10000 )

# they do this in several tutorials..
qsub.raw = qsub.copy()
#qsub.X = scipy.sparse.csr_array(qsub.X)
qsub.obs['Fine_Grained_Label'] = qsub.obs['Fine_Grained_Label'].astype('category')
qsub.obs['Coarse_Grained_Label'] = qsub.obs['Coarse_Grained_Label'].astype('category')
qsub.obs['leiden'] = qsub.obs['leiden'].astype('category')


In [6]:
qsub

AnnData object with n_obs × n_vars = 50076 × 35606
    obs: 'samplename', 'n_genes', 'n_molecules', 'doublet_score', 'percent_mito', 'leiden', 'louvain', 'diagnosis', 'phase', 'sample_diagnosis', 'patient', 'treatment', 'procedure', 'hcl_refined', 'hcl_celltype', 'hcl_score', 'CLid', 'CL_name', 'nobatch_leiden', 'nobatch_louvain', 'cnv_avg', 'has_cnv', 'allcells_leiden', 'allcells_nobatch_leiden', 'ct_leiden', 'ct_nobatch_leiden', 'celltype_split', 'Coarse_Grained_Label', 'Fine_Grained_Label'
    var: 'gene_ids', 'feature_types', 'genome', 'is_mito', 'is_ribo'
    uns: 'leiden', 'leiden_colors', 'leiden_sizes', 'log_X', 'log_raw.X', 'louvain', 'neighbors', 'nobatch', 'nobatch_leiden_colors', 'nobatch_leiden_sizes', 'nobatch_rank_genes_groups', 'nobatch_rank_genes_groups_filtered', 'nobatch_rank_genes_groups_unfiltered', 'paga', 'pca', 'rank_genes_groups', 'rank_genes_groups_filtered', 'rank_genes_groups_unfiltered', 'umap'
    obsm: 'X_pca', 'X_pca_original', 'X_umap', 'X_umap_nobatch'

In [7]:
qsub[qsub.obs.Fine_Grained_Label == 'Neutrophils', "CXCL8"].X.toarray().mean()

C:\Users\dgibbs\CRUK_Atlas_Derived_Data\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


683.4464

In [8]:
qsub[qsub.obs.Fine_Grained_Label == 'Macrophages', "CXCR1"].X

<771x1 sparse matrix of type '<class 'numpy.float32'>'
	with 2 stored elements in Compressed Sparse Row format>

In [9]:
7/771

0.009079118028534372

In [ ]:

res_coarse = sq.gr.ligrec(
    qsub,
    n_perms=1000,
    threshold=0.009,
    cluster_key="Coarse_Grained_Label",
    copy=True,
    use_raw=False,
    interactions_params={"resources": "CellPhoneDB"},
    #interactions=lrs,
    jobs=4
    #transmitter_params={"categories": "ligand"},
    #receiver_params={"categories": "receptor"},
)

res_fine = sq.gr.ligrec(
    qsub,
    n_perms=1000,
    threshold=0.009,
    cluster_key="Fine_Grained_Label",
    copy=True,
    use_raw=False,
    interactions_params={"resources": "CellPhoneDB"},
    #interactions=lrs,
    jobs=4
    #transmitter_params={"categories": "ligand"},
    #receiver_params={"categories": "receptor"},
)


p_coarse = pkl.dumps(res_coarse)
fout = open('results/'+diagstr+'coarse_ligrec_009.pkl', 'wb')
fout.write(p_coarse)
fout.close()

p_fine = pkl.dumps(res_fine)
fout = open('results/'+diagstr+'fine_ligrec_009.pkl', 'wb')
fout.write(p_fine)
fout.close()


C:\Users\dgibbs\CRUK_Atlas_Derived_Data\lib\site-packages\squidpy\gr\_utils.py:141: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(adata.obs[key]):
C:\Users\dgibbs\CRUK_Atlas_Derived_Data\lib\site-packages\squidpy\gr\_ligrec.py:732: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = data.groupby("clusters")


  0%|          | 0/1000 [00:00<?, ?permutation/s]